# Hop ranges and labels (GFQL/Python)

Small, self-contained demo showing bounded hop traversal and optional hop labels on a toy graph.

In [1]:
import pandas as pd
import graphistry

edges = pd.DataFrame({'s': ['a', 'b', 'c'], 'd': ['b', 'c', 'd']})
nodes = pd.DataFrame({'id': ['a', 'b', 'c', 'd']})
g = graphistry.edges(edges, 's', 'd').nodes(nodes, 'id')

# Bounded traversal 2 hops from 'a' with optional labels
result = g.hop(
    nodes=pd.DataFrame({g._node: ['a']}),
    min_hops=2,
    max_hops=2,
    label_node_hops='hop',
    label_edge_hops='edge_hop',
    label_seeds=True
)

print('Nodes with hop labels:')
print(result._nodes.sort_values('hop'))
print('
Edges with hop labels:')
print(result._edges.sort_values('edge_hop'))
